In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.cm
from matplotlib import gridspec
import matplotlib as mpl
import seaborn as sns
from math import ceil, floor
 
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Data description
# num_connections: Number of connections in the range of zipcodes
# num_active_connections: num_connections * perc_of_active_connections
# delivery_perc: percentage of the net consumption of electricity or gas. The lower, the more energy was given back to the grid (for example if you have solar panels)
# perc_of_active_connections: Percentage of active connections in the zipcode range
# annual_consume: Annual consume. Kwh for electricity, m3 for gas
# annual_consume_lowtarif_perc: Percentage of consume during the low tarif hours. From 10 p.m. to 7 a.m. and during weekends.
# smartmeter_perc: percentage of smartmeters in the zipcode ranges
# net_annual_consumption : annual_consume * delivery_perc
# self_production : annual_consume - net_annual_consumption
# self_prod_perc: self_production / annual_consume


In [ ]:
# My work is very easy, let have a see.
# In this page, I will get an example of how to plot a picture and connect this by using Echarts.
# Let's GO!
def load_and_reindex(path,filelist):
    start_time = datetime.now()
    df = None
    for file in filelist:
        year = file[-8:-4]
        manager = file.split('_')[0]
        if df is None:
            df = pd.read_csv(path+file)
            df['year'] = year
            df.index = manager+'_'+year+'_'+df.index.astype(str)
        else:
            temp = pd.read_csv(path+file)
            temp['year'] = year
            temp.index = manager+'_'+year+'_'+temp.index.astype(str)
            df = df.append(temp)
    # adding columns of interest
    df['low_tarif_consumption'] = df['annual_consume'].multiply(df['annual_consume_lowtarif_perc']/100)
    df['num_active_connections'] = df['num_connections'].multiply(df['perc_of_active_connections']/100).astype(int)
    try:
        df['num_smartmeters'] = df['num_connections'].multiply(df['smartmeter_perc']/100).astype(int)
    except ValueError:
        df['num_smartmeters'] = df['num_connections'].multiply(df['smartmeter_perc']/100)
        #print('Number of smartmeters could not be calculated')
    df['net_annual_consumption'] = df['annual_consume'].multiply(df['delivery_perc']/100)
    df['self_production'] = df['annual_consume'] - df['net_annual_consumption']
    df['self_prod_perc'] = df['self_production'].divide(df['annual_consume']/100)
    
    time_elapsed = datetime.now() - start_time
    print('Made main dataframe, time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
    return(df)

In [ ]:
# At the moment, I will analysis the electricity data first.
path = '../input/dutch-energy/Electricity/'
files_all = [f for f in os.listdir(path)]
elec_all = load_and_reindex(path,files_all)

In [ ]:
# make pivot tables of relevant parameter such that we have total per city per year
annual_consume = pd.pivot_table(elec_all,values='annual_consume',index='city',columns='year',aggfunc=np.sum)
num_connections = pd.pivot_table(elec_all,values='num_connections',index='city',columns='year',aggfunc=np.sum)
num_active_connections = pd.pivot_table(elec_all,values='num_active_connections',index='city',columns='year',aggfunc=np.sum)
perc_active_connections = pd.pivot_table(elec_all,values='perc_of_active_connections',index='city',columns='year',aggfunc=np.mean)
smartmeter_perc = pd.pivot_table(elec_all,values='smartmeter_perc',index='city',columns='year',aggfunc=np.mean)
smartmeter_perc_median = pd.pivot_table(elec_all,values='smartmeter_perc',index='city',columns='year',aggfunc=np.median)
num_smartmeters = pd.pivot_table(elec_all,values='num_smartmeters',index='city',columns='year',aggfunc=np.sum)
self_production = pd.pivot_table(elec_all,values='self_production',index='city',columns='year',aggfunc=np.sum)
self_prod_perc_mean = pd.pivot_table(elec_all,values='self_prod_perc',index='city',columns='year',aggfunc=np.mean)
net_annu_consume = pd.pivot_table(elec_all,values='net_annual_consumption',index='city',columns='year',aggfunc=np.sum)
low_tarif_consumption = pd.pivot_table(elec_all,values='low_tarif_consumption',index='city',columns='year',aggfunc=np.sum)
annu_cons_lowtarif_perc = pd.pivot_table(elec_all,values='annual_consume_lowtarif_perc',index='city',columns='year',aggfunc=np.mean)

In [ ]:
# We do not use the year of 2009,because the data is not incorrect.
annual_consume.drop('2009',axis=1,inplace=True)
num_connections.drop('2009',axis=1,inplace=True)
num_active_connections.drop('2009',axis=1,inplace=True)
perc_active_connections.drop('2009',axis=1,inplace=True)
smartmeter_perc.drop('2009',axis=1,inplace=True)
num_smartmeters.drop('2009',axis=1,inplace=True)
self_production.drop('2009',axis=1,inplace=True)
net_annu_consume.drop('2009',axis=1,inplace=True)
low_tarif_consumption.drop('2009',axis=1,inplace=True)
annu_cons_lowtarif_perc.drop('2009',axis=1,inplace=True)

In [ ]:
# Here is an examole of annual elec consumption.
# I use the value of 'annual_consume.sum()',to make a picture by using Echarts.
# Choose a suitable form to plot it, and download it as a HTML document, and then modify it by using JAVA script.
# Finally, Putting it online.
num_smartmeters.sum()